# Clean data: users, jobs, user_history, apps
by: Sam

- Input: jobs.tsv, users.tsv, user_history.tsv, apps.tsv
- No duplication in data
- Output:
    - JOBS: - jobs_clean.csv: only clean HTML tags
            - jobset_clean.csv: clean HTML tags + drop null values in Tile, Description, Requirement
    - USERS: users_clean.csv: drop null value in "Major", "TotalYearsExperience", "CurrentlyEmployed"
    - USER_HISTORY: work_history.csv: Filter only for user_id in the users list, drop null value in "Job Title".
    - APPS: - warm_exp_users.csv: users with user_id in work_history & in application data
            - warm_exp_apps.csv: filter applicaton only for warm_exp_user
    

In [1]:
import pandas as pd
import numpy as np

In [2]:
import os
cwd = os.getcwd()
print("Working directory:", cwd)

Working directory: /Users/anhtth/Library/CloudStorage/OneDrive-UniversityofTwente/2023 UT- THESIS/1-Code/0.cb12_main


In [3]:
path = './data_raw/'

## Load raw data

In [4]:
users = pd.read_csv(path+'users.tsv', sep='\t', 
                    error_bad_lines=False, 
                    #nrows=99999
                   )

/var/folders/rt/sq9c2fwj2wz441h13l9hcfrc0000gn/T/ipykernel_90621/512833297.py:1: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  users = pd.read_csv(path+'users.tsv', sep='\t',


In [5]:
user_history = pd.read_csv(path+'user_history.tsv', sep='\t', 
                           error_bad_lines=False, 
                           #nrows=99999
                          )

/var/folders/rt/sq9c2fwj2wz441h13l9hcfrc0000gn/T/ipykernel_90621/2900069451.py:1: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  user_history = pd.read_csv(path+'user_history.tsv', sep='\t',


In [6]:
jobs = pd.read_csv(path+'jobs.tsv', sep='\t', error_bad_lines=False, 
                   #nrows=99999
                  )

/var/folders/rt/sq9c2fwj2wz441h13l9hcfrc0000gn/T/ipykernel_90621/2359911626.py:1: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  jobs = pd.read_csv(path+'jobs.tsv', sep='\t', error_bad_lines=False,
b'Skipping line 122433: expected 11 fields, saw 12\n'
b'Skipping line 602576: expected 11 fields, saw 12\n'
b'Skipping line 990950: expected 11 fields, saw 12\n'
/var/folders/rt/sq9c2fwj2wz441h13l9hcfrc0000gn/T/ipykernel_90621/2359911626.py:1: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  jobs = pd.read_csv(path+'jobs.tsv', sep='\t', error_bad_lines=False,


In [7]:
apps = pd.read_csv(path+'apps.tsv', sep='\t', error_bad_lines=False)

/var/folders/rt/sq9c2fwj2wz441h13l9hcfrc0000gn/T/ipykernel_90621/1816833552.py:1: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  apps = pd.read_csv(path+'apps.tsv', sep='\t', error_bad_lines=False)


### UDF - Cleaning HTML tags:

3 user-defined functions based on different repositories
- filter_tags(): Repo PJFNN
- preprocessor(): Repo job-recommender: clean text are lower case and not contain punctuation
- clean_html(): Repo Liac2022 => Similar to filter_tags. Clean text contain capital letters and punctuation

Test the udf on description, title, and requirement for comparison
Measure similarity between outputs

In [8]:
# Ref: Repo PJFNN

import re

def filter_tags(htmlstr):

    re_cdata=re.compile('//<!\[CDATA\[[^>]*//\]\]>',re.I) 
    re_script=re.compile('<\s*script[^>]*>[^<]*<\s*/\s*script\s*>',re.I)#Script
    re_style=re.compile('<\s*style[^>]*>[^<]*<\s*/\s*style\s*>',re.I)#style
    re_br=re.compile('<br\s*?/?>')
    re_h=re.compile('</?\w+[^>]*>')
    re_comment=re.compile('<!--[^>]*-->')
    s=re_cdata.sub('',htmlstr)
    s=re_script.sub('',s) 
    s=re_style.sub('',s)
    s=re_br.sub('\n',s)
    s=re_h.sub('',s) 
    s=re_comment.sub('',s)
    blank_line=re.compile('\n+')
    s=blank_line.sub('\n',s)
    s=s.replace('\\r'," ")
    s=s.replace('\\t'," ")
    s=s.replace('\n'," ")
    s=s.replace('\\n'," ")
    s = re.sub(r'(https|http)?:\/\/(\w|\.|\/|\?|\=|\&|\%)*\b', '', s, flags=re.MULTILINE)
    s = re.sub(r'[\w\-_]+(\.[\w\-_]+)+([\w\-\.,@?^=%&amp;:/~\+#]*[\w\-\@?^=%&amp;/~\+#])?', '', s, flags=re.MULTILINE)
    s = re.sub(r'(www)?:\/\/(\w|\.|\/|\?|\=|\&|\%)*\b', '', s, flags=re.MULTILINE)
    s = re.sub(r'[0-9a-zA-Z.]+@[0-9a-zA-Z.]', " ", s, flags=re.MULTILINE)
    s = re.sub('\xa0', " ", s, flags=re.MULTILINE)
    s=replaceCharEntity(s)
    return s


def replaceCharEntity(htmlstr):
    CHAR_ENTITIES={'nbsp':' ','160':' ',
        'lt':'<','60':'<',
        'gt':'>','62':'>',
        'amp':'&','38':'&',
        'quot':'"','34':'"',}
   
    re_charEntity=re.compile(r'&#?(?P<name>\w+);')
    sz=re_charEntity.search(htmlstr)
    while sz:
        entity=sz.group()
        key=sz.group('name')
        try:
            htmlstr=re_charEntity.sub(CHAR_ENTITIES[key],htmlstr,1)
            sz=re_charEntity.search(htmlstr)
        except KeyError:
            htmlstr=re_charEntity.sub('',htmlstr,1)
            sz=re_charEntity.search(htmlstr)
    return htmlstr
def repalce(s,re_exp,repl_string):
    return re_exp.sub(repl_string,s)

In [9]:
jobs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1091923 entries, 0 to 1091922
Data columns (total 11 columns):
 #   Column        Non-Null Count    Dtype 
---  ------        --------------    ----- 
 0   JobID         1091923 non-null  int64 
 1   WindowID      1091923 non-null  int64 
 2   Title         1091916 non-null  object
 3   Description   1091913 non-null  object
 4   Requirements  1050521 non-null  object
 5   City          1091921 non-null  object
 6   State         1091922 non-null  object
 7   Country       1091920 non-null  object
 8   Zip5          685724 non-null   object
 9   StartDate     1091923 non-null  object
 10  EndDate       1091914 non-null  object
dtypes: int64(2), object(9)
memory usage: 91.6+ MB


In [10]:
test_desc = jobs.Description[0]

In [11]:
test_desc

'<p>Security Clearance Required:&nbsp; Top Secret </p>\\r<p>Job Number: TMR-447</p>\\r<p>Location of Job:&nbsp; Washington, DC</p>\\r<p>TMR, Inc. is an Equal Employment Opportunity Company</p>\\r<p>For more job opportunities with TMR, visit our website <a href="http://www.tmrhq.com/">www.tmrhq.com</a></p>\\r<p>Send Resumes to HR@tmrhq2.com</p>\\r<p>&nbsp;</p>\\r<p>JOB SUMMARY:</p>\\r<p>&nbsp;</p>\\r<p>Leads the customer&rsquo;s overall Cyber Security strategy, formalizes service offerings consisted with ITIL best practices, and provides design and architecture support.</p>\\r<p>&nbsp;</p>\\r<ul>\\r    <li>Provide security design / architecture support for OJP&rsquo;s IT Security Division (ITSD) </li>\\r    <li>Leads the SECOPS team in the day to day OJP Security Operations support&nbsp; </li>\\r    <li>Provides direction when needed in a security incident or technical issues </li>\\r    <li>Works in concert with network operations on design /integration for best security posture</li>\\

In [12]:
filter_tags(test_desc)

'Security Clearance Required:  Top Secret  Job Number: TMR-447 Location of Job:  Washington, DC TMR, Inc. is an Equal Employment Opportunity Company For more job opportunities with TMR, visit our website  Send Resumes to HR@   JOB SUMMARY:   Leads the customers overall Cyber Security strategy, formalizes service offerings consisted with ITIL best practices, and provides design and architecture support.        Provide security design / architecture support for OJPs IT Security Division (ITSD)      Leads the SECOPS team in the day to day OJP Security Operations support       Provides direction when needed in a security incident or technical issues      Works in concert with network operations on design /integration for best security posture     Supports business development functions including Capture Management, Proposal Development and responses, and other initiatives to include conferences, trade shows, webinars, developing white papers and the like.     Identifies resources and mento

In [13]:
# Ref: job-recommender

import re

def preprocessor(text):
    text = text.replace('\\r', '').replace('&nbsp', '').replace('\n', '')
    text = re.sub('<[^>]*>', '', text)
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', text)
    text = re.sub('[\W]+', ' ', text.lower()) +\
        ' '.join(emoticons).replace('-', '')
    return text

In [14]:
preprocessor(test_desc)

'security clearance required top secret job number tmr 447location of job washington dctmr inc is an equal employment opportunity companyfor more job opportunities with tmr visit our website www tmrhq comsend resumes to hr tmrhq2 com job summary leads the customer rsquo s overall cyber security strategy formalizes service offerings consisted with itil best practices and provides design and architecture support provide security design architecture support for ojp rsquo s it security division itsd leads the secops team in the day to day ojp security operations support provides direction when needed in a security incident or technical issues works in concert with network operations on design integration for best security posture supports business development functions including capture management proposal development and responses and other initiatives to include conferences trade shows webinars developing white papers and the like identifies resources and mentors in house talent to ensur

In [15]:
# REPO liac 2022
import re
def clean_html(x):
    x = re.sub('<[^<]+?>', '', x)
    x = re.sub('\\\\r', '', x)
    x = re.sub('\\\\n', '', x)
    x = re.sub('&nbsp;', ' ', x)
    x = re.sub('[—]+', ' ', x)
    x = re.sub('/', ' ', x)
    return x


In [16]:
clean_html(test_desc)

'Security Clearance Required:  Top Secret Job Number: TMR-447Location of Job:  Washington, DCTMR, Inc. is an Equal Employment Opportunity CompanyFor more job opportunities with TMR, visit our website www.tmrhq.comSend Resumes to HR@tmrhq2.com JOB SUMMARY: Leads the customer&rsquo;s overall Cyber Security strategy, formalizes service offerings consisted with ITIL best practices, and provides design and architecture support.     Provide security design   architecture support for OJP&rsquo;s IT Security Division (ITSD)     Leads the SECOPS team in the day to day OJP Security Operations support      Provides direction when needed in a security incident or technical issues     Works in concert with network operations on design  integration for best security posture    Supports business development functions including Capture Management, Proposal Development and responses, and other initiatives to include conferences, trade shows, webinars, developing white papers and the like.    Identifies

- Test on Job Requirements

In [17]:
test_req = jobs.Requirements[0]

In [18]:
test_req

'<p>SKILL SET</p>\\r<p>&nbsp;</p>\\r<p>Network Security tools:</p>\\r<p>&nbsp;</p>\\r<p>Webdefend Web Application Firewall (WAF), Cisco Routers, Fortigate 3800 Firewall series, Palo Alto 4000 firewall series, Cisco ASA 5xx Firewall Platform, Cisco&nbsp; FWSM,&nbsp; SourceFire Defense Center, SourceFire IP Sensor Platform, BlueCoat SG Appliance, F5 BigIP(reverse proxy).</p>\\r<p>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; </p>\\r<p>Web Application tools:&nbsp; </p>\\r<p>&nbsp;</p>\\r<p>AppDective, Fortify SCA, HP WebInspect, and the like.</p>\\r<p>&nbsp;</p>\\r<p>Network Vulnerability tools:&nbsp; </p>\\r<p>&nbsp;</p>\\r<p>Tenable Security Center, McAfee Foundstone scanner, Cain and Able, L0phtcrack - Password Cracker, Nessus Vulnerability Scanner, NMAP &ndash; Port Scanner, and other scanning and vulnerability mapping tools.&nbsp; </p>\\r<p>&nbsp;</p>\\r<p>&nbsp;</p>\\r<p>DESIRABLE SKILLS:</p>\\r<p>&nbsp;</p>\\r<p>CISSP and/or related Certifications</p>\\r<p>&nbs

In [19]:
# Repo 1: filter_tags()
filter_tags(test_req)

'SKILL SET   Network Security tools:   Webdefend Web Application Firewall (WAF), Cisco Routers, Fortigate 3800 Firewall series, Palo Alto 4000 firewall series, Cisco ASA 5xx Firewall Platform, Cisco  FWSM,  SourceFire Defense Center, SourceFire IP Sensor Platform, BlueCoat SG Appliance, F5 BigIP(reverse proxy).              Web Application tools:     AppDective, Fortify SCA, HP WebInspect, and the like.   Network Vulnerability tools:     Tenable Security Center, McAfee Foundstone scanner, Cain and Able, L0phtcrack - Password Cracker, Nessus Vulnerability Scanner, NMAP  Port Scanner, and other scanning and vulnerability mapping tools.       DESIRABLE SKILLS:   CISSP and/or related Certifications   EDUCATION AND YEARS OF EXPERIENCE:   BS Computer Science or related discipline; minimum of 8 years in IT Security; minimum 4 years in Senior/Lead position   APPLY HERE'

In [20]:
# Repo 2: preprocessor()
preprocessor(test_req)

'skill set network security tools webdefend web application firewall waf cisco routers fortigate 3800 firewall series palo alto 4000 firewall series cisco asa 5xx firewall platform cisco fwsm sourcefire defense center sourcefire ip sensor platform bluecoat sg appliance f5 bigip reverse proxy web application tools appdective fortify sca hp webinspect and the like network vulnerability tools tenable security center mcafee foundstone scanner cain and able l0phtcrack password cracker nessus vulnerability scanner nmap ndash port scanner and other scanning and vulnerability mapping tools desirable skills cissp and or related certifications education and years of experience bs computer science or related discipline minimum of 8 years in it security minimum 4 years in senior lead position apply here;D'

In [21]:
# Repo 3: clean_html()
clean_html(test_req)

'SKILL SET Network Security tools: Webdefend Web Application Firewall (WAF), Cisco Routers, Fortigate 3800 Firewall series, Palo Alto 4000 firewall series, Cisco ASA 5xx Firewall Platform, Cisco  FWSM,  SourceFire Defense Center, SourceFire IP Sensor Platform, BlueCoat SG Appliance, F5 BigIP(reverse proxy).            Web Application tools:   AppDective, Fortify SCA, HP WebInspect, and the like. Network Vulnerability tools:   Tenable Security Center, McAfee Foundstone scanner, Cain and Able, L0phtcrack - Password Cracker, Nessus Vulnerability Scanner, NMAP &ndash; Port Scanner, and other scanning and vulnerability mapping tools.    DESIRABLE SKILLS: CISSP and or related Certifications EDUCATION AND YEARS OF EXPERIENCE: BS Computer Science or related discipline; minimum of 8 years in IT Security; minimum 4 years in Senior Lead position APPLY HERE'

- Test on Job Title

In [22]:
test_tit = jobs.Title[0]

In [23]:
# Repo 1: filter_tags()
filter_tags(test_tit)

'Security Engineer/Technical Lead'

In [24]:
# Repo 2: preprocessor()
preprocessor(test_tit)

'security engineer technical lead'

In [25]:
# Repo 3: clean_html()
clean_html(test_tit)

'Security Engineer Technical Lead'

In [26]:
test_user_hist = user_history.JobTitle[22]

In [27]:
user_history.JobTitle[25]

'Student Assistant'

In [28]:
test_user_hist

'Data Analyst'

In [29]:
import gensim
import spacy
#nlp = spacy.load('en_core_web_lg')
import en_core_web_sm
nlp = en_core_web_sm.load()

In [30]:
# test_title after cleaning tags: Security Engineer Technical Lead
# test title by user history: 'Student Assistant'
nlp(clean_html(test_tit)).similarity(nlp(user_history.JobTitle[25]))

/var/folders/rt/sq9c2fwj2wz441h13l9hcfrc0000gn/T/ipykernel_90621/84402056.py:3: UserWarning: [W007] The model you're using has no word vectors loaded, so the result of the Doc.similarity method will be based on the tagger, parser and NER, which may not give useful similarity judgements. This may happen if you're using one of the small models, e.g. `en_core_web_sm`, which don't ship with word vectors and only use context-sensitive tensors. You can always add your own word vectors, or use one of the larger models instead if available.
  nlp(clean_html(test_tit)).similarity(nlp(user_history.JobTitle[25]))


0.7542843077825117

In [31]:
# test_title after cleaning tags: Security Engineer Technical Lead
# test title by user history: 'Data Analyst'
nlp(clean_html(test_tit)).similarity(nlp(test_user_hist))

/var/folders/rt/sq9c2fwj2wz441h13l9hcfrc0000gn/T/ipykernel_90621/1774482677.py:3: UserWarning: [W007] The model you're using has no word vectors loaded, so the result of the Doc.similarity method will be based on the tagger, parser and NER, which may not give useful similarity judgements. This may happen if you're using one of the small models, e.g. `en_core_web_sm`, which don't ship with word vectors and only use context-sensitive tensors. You can always add your own word vectors, or use one of the larger models instead if available.
  nlp(clean_html(test_tit)).similarity(nlp(test_user_hist))


0.8205649080366024

In [32]:
# test_title after cleaning tags: 'security engineer technical lead'
# test title by user history: 'Data Analyst'
nlp(preprocessor(test_tit)).similarity(nlp(test_user_hist))

/var/folders/rt/sq9c2fwj2wz441h13l9hcfrc0000gn/T/ipykernel_90621/3030846139.py:3: UserWarning: [W007] The model you're using has no word vectors loaded, so the result of the Doc.similarity method will be based on the tagger, parser and NER, which may not give useful similarity judgements. This may happen if you're using one of the small models, e.g. `en_core_web_sm`, which don't ship with word vectors and only use context-sensitive tensors. You can always add your own word vectors, or use one of the larger models instead if available.
  nlp(preprocessor(test_tit)).similarity(nlp(test_user_hist))


0.6266870048515475

In [33]:
test_user_hist.lower()

'data analyst'

In [34]:
# test_title after cleaning tags: 'security engineer technical lead'
# test title by user history: 'data analyst'
nlp(preprocessor(test_tit)).similarity(nlp(test_user_hist.lower()))

/var/folders/rt/sq9c2fwj2wz441h13l9hcfrc0000gn/T/ipykernel_90621/1135489744.py:3: UserWarning: [W007] The model you're using has no word vectors loaded, so the result of the Doc.similarity method will be based on the tagger, parser and NER, which may not give useful similarity judgements. This may happen if you're using one of the small models, e.g. `en_core_web_sm`, which don't ship with word vectors and only use context-sensitive tensors. You can always add your own word vectors, or use one of the larger models instead if available.
  nlp(preprocessor(test_tit)).similarity(nlp(test_user_hist.lower()))


0.7333151176871748

## Clean jobs

Output: 
- jobs_clean.csv
- jobset_clean.csv

### Check null data in jobs

In [35]:
jobs.isnull().values.any()

True

In [36]:
jobs['Title'].isnull().values.sum()

7

In [37]:
jobs['Description'].isnull().values.sum()

10

In [38]:
jobs['Requirements'].isnull().values.sum()

41402

In [39]:
jobs[jobs['Requirements'].isnull()]

,JobID,WindowID,Title,Description,Requirements,City,State,Country,Zip5,StartDate,EndDate
753,2777,1,Information Assurance Officer,"Knowledge, Skills & Abilities\r\n\r\nCommunica...",NaN,Arlington,VA,US,22202,2012-03-14 16:17:36.483,2012-04-13 23:59:59
754,2778,1,IT Comsec / Network Engineer Sr Staff,The selected individual will provide day-to-da...,NaN,San Antonio,TX,US,78216,2012-03-14 16:17:39.223,2012-04-13 23:59:59
769,2847,1,Senior Project Engineer,The project engineer has single-point accounta...,NaN,Gainesville,VA,US,20155,2012-03-21 20:45:56.607,2012-04-20 23:59:59
1299,5354,1,"Manager, eBP Loss Prevention","<p><strong>Manager, Loss Prevention - eBay Buy...",NaN,San Jose,CA,US,95125,2012-03-07 12:41:47.16,2012-04-06 23:59:59
1302,5357,1,Information Security Architect,"<p>X.commerce is a startup within eBay, Inc. a...",NaN,Austin,TX,US,78750,2012-03-08 15:01:17.27,2012-04-07 23:59:59
...,...,...,...,...,...,...,...,...,...,...,...
1069382,896350,7,ASSISTANT ENTRY LEVEL,"<div style=""text-align: center""><strong>CUSTOM...",NaN,Palmyra,NJ,US,08065,2012-06-25 18:23:56.77,2012-07-24 23:59:59
1075449,953790,7,Restaurant manager,The manager is responsible for leading departm...,NaN,Chillicothe,OH,US,NaN,2012-05-25 18:37:23.457,2012-06-24 23:59:00
1075450,953792,7,Restaurant Manager,The manager is responsible for leading departm...,NaN,Houston,TX,US,NaN,2012-05-27 12:00:12.5,2012-06-26 23:59:00
1081290,1010874,7,Business Unit Manager-Paint,<div><strong>Business Unit Manager-Paint </str...,NaN,Macungie,PA,US,18062,2012-06-29 14:36:29.927,2012-07-28 23:59:59


In [40]:
job_set = jobs.dropna(axis=0,subset=["Description","Requirements","Title"])

In [41]:
job_set.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1050509 entries, 0 to 1091922
Data columns (total 11 columns):
 #   Column        Non-Null Count    Dtype 
---  ------        --------------    ----- 
 0   JobID         1050509 non-null  int64 
 1   WindowID      1050509 non-null  int64 
 2   Title         1050509 non-null  object
 3   Description   1050509 non-null  object
 4   Requirements  1050509 non-null  object
 5   City          1050507 non-null  object
 6   State         1050508 non-null  object
 7   Country       1050506 non-null  object
 8   Zip5          658844 non-null   object
 9   StartDate     1050509 non-null  object
 10  EndDate       1050500 non-null  object
dtypes: int64(2), object(9)
memory usage: 96.2+ MB


### Clean html tags in jobs: using clean_html()

In [42]:
jobs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1091923 entries, 0 to 1091922
Data columns (total 11 columns):
 #   Column        Non-Null Count    Dtype 
---  ------        --------------    ----- 
 0   JobID         1091923 non-null  int64 
 1   WindowID      1091923 non-null  int64 
 2   Title         1091916 non-null  object
 3   Description   1091913 non-null  object
 4   Requirements  1050521 non-null  object
 5   City          1091921 non-null  object
 6   State         1091922 non-null  object
 7   Country       1091920 non-null  object
 8   Zip5          685724 non-null   object
 9   StartDate     1091923 non-null  object
 10  EndDate       1091914 non-null  object
dtypes: int64(2), object(9)
memory usage: 91.6+ MB


In [43]:
jobs['Title'] = jobs['Title'].astype(dtype='str').apply(clean_html)

In [44]:
jobs['Description'] = jobs['Description'].astype(dtype='str').apply(clean_html)

In [45]:
jobs['Requirements'] = jobs['Requirements'].astype(dtype='str').apply(clean_html)

In [46]:
jobs.head()

,JobID,WindowID,Title,Description,Requirements,City,State,Country,Zip5,StartDate,EndDate
0,1,1,Security Engineer Technical Lead,Security Clearance Required: Top Secret Job N...,SKILL SET Network Security tools: Webdefend We...,Washington,DC,US,20531,2012-03-07 13:17:01.643,2012-04-06 23:59:59
1,4,1,SAP Business Analyst WM,NO Corp. to Corp resumes are being considered ...,WHAT YOU NEED: Four year college degreeMinimum...,Charlotte,NC,US,28217,2012-03-21 02:03:44.137,2012-04-20 23:59:59
2,7,1,P T HUMAN RESOURCES ASSISTANT,P T HUMAN RESOURCES ASSISTANT 1-2 ye...,Please refer to the Job Description to view th...,Winter Park,FL,US,32792,2012-03-02 16:36:55.447,2012-04-01 23:59:59
3,8,1,Route Delivery Drivers,CITY BEVERAGES Come to work for the best in th...,Please refer to the Job Description to view th...,Orlando,FL,US,NaN,2012-03-03 09:01:10.077,2012-04-02 23:59:59
4,9,1,Housekeeping,I make sure every part of their day is magica...,Please refer to the Job Description to view th...,Orlando,FL,US,NaN,2012-03-03 09:01:11.88,2012-04-02 23:59:59


In [47]:
jobs.to_csv('./data_processed/jobs_clean.csv', header=True, index=False)

### Clean html tags in job_set: using clean_html()

In [48]:
job_set['Title'] = job_set['Title'].astype(dtype='str').apply(clean_html)
job_set['Description'] = job_set['Description'].astype(dtype='str').apply(clean_html)
job_set['Requirements'] = job_set['Requirements'].astype(dtype='str').apply(clean_html)


/var/folders/rt/sq9c2fwj2wz441h13l9hcfrc0000gn/T/ipykernel_90621/1042621947.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  job_set['Title'] = job_set['Title'].astype(dtype='str').apply(clean_html)
/var/folders/rt/sq9c2fwj2wz441h13l9hcfrc0000gn/T/ipykernel_90621/1042621947.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  job_set['Description'] = job_set['Description'].astype(dtype='str').apply(clean_html)
/var/folders/rt/sq9c2fwj2wz441h13l9hcfrc0000gn/T/ipykernel_90621/1042621947.py:3: SettingWithCop

In [49]:
job_set.to_csv('./data_processed/jobset_clean.csv', header=True, index=False)

## Clean users data 
Output: 
- user_clean.csv
- warm_exp_apps.csv


In [50]:
users.isnull().values.any()

True

In [51]:
users.isnull().sum().sum()

275951

In [52]:
users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 389708 entries, 0 to 389707
Data columns (total 15 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   UserID                389708 non-null  int64  
 1   WindowID              389708 non-null  int64  
 2   Split                 389708 non-null  object 
 3   City                  389708 non-null  object 
 4   State                 389218 non-null  object 
 5   Country               389708 non-null  object 
 6   ZipCode               387974 non-null  object 
 7   DegreeType            389708 non-null  object 
 8   Major                 292468 non-null  object 
 9   GraduationDate        269477 non-null  object 
 10  WorkHistoryCount      389708 non-null  int64  
 11  TotalYearsExperience  375528 non-null  float64
 12  CurrentlyEmployed     347632 non-null  object 
 13  ManagedOthers         389708 non-null  object 
 14  ManagedHowMany        389708 non-null  int64  
dtype

In [53]:
users[users['Major'].isnull()]

,UserID,WindowID,Split,City,State,Country,ZipCode,DegreeType,Major,GraduationDate,WorkHistoryCount,TotalYearsExperience,CurrentlyEmployed,ManagedOthers,ManagedHowMany
0,47,1,Train,Paramount,CA,US,90723,High School,NaN,1999-06-01 00:00:00,3,10.0,Yes,No,0
11,411,1,Train,Lutz,FL,US,33559,None,NaN,NaN,3,11.0,Yes,Yes,1
16,501,1,Train,Cape Coral,FL,US,33990,Bachelor's,NaN,NaN,4,37.0,No,No,0
20,553,1,Train,Debary,FL,US,32713,Bachelor's,NaN,NaN,3,10.0,NaN,No,0
23,589,1,Train,Ventura,CA,US,93003,Associate's,NaN,1997-06-01 00:00:00,5,26.0,No,Yes,75
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
389697,1471828,7,Train,Atlanta,GA,US,30312,PhD,NaN,2006-05-01 00:00:00,3,6.0,Yes,No,0
389701,1471901,7,Train,Port Clinton,OH,US,43452,None,NaN,NaN,7,25.0,No,No,0
389702,1471975,7,Train,Concord,NC,US,28027,None,NaN,NaN,5,14.0,Yes,No,0
389704,1472042,7,Train,Saint Clair Shores,MI,US,48081,High School,NaN,1970-01-01 00:00:00,7,32.0,Yes,No,0


In [54]:
users['Major'].isnull().values.sum()

97240

In [55]:
users['TotalYearsExperience'].isnull().values.sum()

14180

In [56]:
users['CurrentlyEmployed'].isnull().values.sum()

42076

In [57]:
# Drop users NA values in major, total years of experience, and currently employed
user_set = users.dropna(axis=0,subset=["Major", "TotalYearsExperience", "CurrentlyEmployed"])

In [58]:
user_set.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 258260 entries, 1 to 389706
Data columns (total 15 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   UserID                258260 non-null  int64  
 1   WindowID              258260 non-null  int64  
 2   Split                 258260 non-null  object 
 3   City                  258260 non-null  object 
 4   State                 257917 non-null  object 
 5   Country               258260 non-null  object 
 6   ZipCode               257094 non-null  object 
 7   DegreeType            258260 non-null  object 
 8   Major                 258260 non-null  object 
 9   GraduationDate        203034 non-null  object 
 10  WorkHistoryCount      258260 non-null  int64  
 11  TotalYearsExperience  258260 non-null  float64
 12  CurrentlyEmployed     258260 non-null  object 
 13  ManagedOthers         258260 non-null  object 
 14  ManagedHowMany        258260 non-null  int64  
dtype

In [59]:
user_set.Split.value_counts(), user_set.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 258260 entries, 1 to 389706
Data columns (total 15 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   UserID                258260 non-null  int64  
 1   WindowID              258260 non-null  int64  
 2   Split                 258260 non-null  object 
 3   City                  258260 non-null  object 
 4   State                 257917 non-null  object 
 5   Country               258260 non-null  object 
 6   ZipCode               257094 non-null  object 
 7   DegreeType            258260 non-null  object 
 8   Major                 258260 non-null  object 
 9   GraduationDate        203034 non-null  object 
 10  WorkHistoryCount      258260 non-null  int64  
 11  TotalYearsExperience  258260 non-null  float64
 12  CurrentlyEmployed     258260 non-null  object 
 13  ManagedOthers         258260 non-null  object 
 14  ManagedHowMany        258260 non-null  int64  
dtype

(Train    242772
 Test      15488
 Name: Split, dtype: int64,
 None)

In [60]:
user_set.UserID.unique().size

258260

In [61]:
# Export dataset to preprocessed
user_set.to_csv('./data_processed/users_clean.csv', header=True, index=False)

In [62]:
user_id = user_set.UserID.unique().tolist()

## Clean user_history
Output: work_history.csv

In [63]:
# Filter work_history for user in the list
work_history = user_history[user_history.UserID.isin(user_id)]

In [64]:
work_history.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1212095 entries, 3 to 1753897
Data columns (total 5 columns):
 #   Column    Non-Null Count    Dtype 
---  ------    --------------    ----- 
 0   UserID    1212095 non-null  int64 
 1   WindowID  1212095 non-null  int64 
 2   Split     1212095 non-null  object
 3   Sequence  1212095 non-null  int64 
 4   JobTitle  1144557 non-null  object
dtypes: int64(3), object(2)
memory usage: 55.5+ MB


In [65]:
work_history['JobTitle'].isnull().values.sum()

67538

In [66]:
work_history[work_history['JobTitle'].isnull()]

,UserID,WindowID,Split,Sequence,JobTitle
6,72,1,Train,4,NaN
10,72,1,Train,8,NaN
51,395,1,Train,4,NaN
56,395,1,Train,9,NaN
57,395,1,Train,10,NaN
...,...,...,...,...,...
1753782,1471550,7,Train,2,NaN
1753785,1471550,7,Train,5,NaN
1753797,1471583,7,Train,3,NaN
1753832,1471656,7,Train,4,NaN


In [67]:
work_history.dropna(axis=0, subset=["JobTitle"], inplace=True)

/var/folders/rt/sq9c2fwj2wz441h13l9hcfrc0000gn/T/ipykernel_90621/2536373953.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  work_history.dropna(axis=0, subset=["JobTitle"], inplace=True)


In [68]:
work_user_id = work_history.UserID.unique()

## Clean apps
Output: warm_apps.csv

In [73]:
job_id = job_set.JobID.unique().tolist()

In [74]:
# Filter applications with user and job id in the two above lists
application_record = apps[(apps.UserID.isin(user_id))&(apps.JobID.isin(job_id))]

In [75]:
work_history.info(), application_record.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1144557 entries, 3 to 1753897
Data columns (total 5 columns):
 #   Column    Non-Null Count    Dtype 
---  ------    --------------    ----- 
 0   UserID    1144557 non-null  int64 
 1   WindowID  1144557 non-null  int64 
 2   Split     1144557 non-null  object
 3   Sequence  1144557 non-null  int64 
 4   JobTitle  1144557 non-null  object
dtypes: int64(3), object(2)
memory usage: 52.4+ MB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 1049043 entries, 8 to 1603098
Data columns (total 5 columns):
 #   Column           Non-Null Count    Dtype 
---  ------           --------------    ----- 
 0   UserID           1049043 non-null  int64 
 1   WindowID         1049043 non-null  int64 
 2   Split            1049043 non-null  object
 3   ApplicationDate  1049043 non-null  object
 4   JobID            1049043 non-null  int64 
dtypes: int64(3), object(2)
memory usage: 48.0+ MB


(None, None)

In [76]:
# user_id from work_history
work_user_id = work_history.UserID.unique()
len(work_user_id)

256021

In [77]:
# user_id from application (warm)
application_user_id = application_record.UserID.unique()
len(application_user_id)

210685

In [78]:
# user_id from clean user data
len(user_id)

258260

In [81]:
# Filter users with experience & appear in application data (warm_exp_user)
warm_exp_user = user_set[(user_set.UserID.isin(work_user_id))&(user_set.UserID.isin(application_user_id))]
warm_exp_user_id = warm_exp_user.UserID.unique()
len(warm_exp_user_id)

208953

In [82]:
# Export warm application
application_record.to_csv('./data_processed/apps_warm.csv', header=True, index=False)

In [83]:
# Filter warm, experience user history and application
application_record = application_record[application_record.UserID.isin(warm_exp_user_id)]
work_history = work_history[work_history.UserID.isin(warm_exp_user_id)]

In [84]:
# Check data for users and application records in training and testing (Column: Split)
application_record.Split.value_counts(), warm_exp_user.Split.value_counts()

(Train    921114
 Test     119556
 Name: Split, dtype: int64,
 Train    200248
 Test       8705
 Name: Split, dtype: int64)

In [85]:
(warm_exp_user.UserID.unique() == application_record.UserID.unique()).all(), (warm_exp_user.UserID.unique() == work_history.UserID.unique()).all()

(True, True)

In [86]:
warm_exp_user.to_csv('./data_processed/warm_exp_users.csv', header=True, index=False)

In [87]:
application_record.to_csv('./data_processed/warm_exp_apps.csv', header=True, index=False)

## Extract clean work_history data

In [1]:
import pandas as pd
import numpy as np

In [3]:
# Load data
path = './data_raw/'
user_set = pd.read_csv('./data_processed/users_clean.csv')
user_id = user_set.UserID.unique().tolist()
user_history = pd.read_csv(path+'user_history.tsv', sep='\t', 
                           error_bad_lines=False, 
                           #nrows=99999
                          )


/var/folders/rt/sq9c2fwj2wz441h13l9hcfrc0000gn/T/ipykernel_1834/3892858650.py:5: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  user_history = pd.read_csv(path+'user_history.tsv', sep='\t',


In [4]:
# Filter work_history for user in the list
work_history = user_history[user_history.UserID.isin(user_id)]

In [5]:
work_history.dropna(axis=0, subset=["JobTitle"], inplace=True)

/var/folders/rt/sq9c2fwj2wz441h13l9hcfrc0000gn/T/ipykernel_1834/2536373953.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  work_history.dropna(axis=0, subset=["JobTitle"], inplace=True)


In [6]:
work_history.to_csv('./data_processed/work_history.csv', header=True, index=False)